In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Modellname
model_name = "Babelscape/wikineural-multilingual-ner"

# Label-Liste und Mapping
label_list = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

# Lade Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Lade Modell mit der Option, Größenunterschiede zu ignorieren
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    ignore_mismatched_sizes=True
)

# Aktualisiere das Modell mit den passenden Labels
model.config.id2label = id2label
model.config.label2id = label2id





Some weights of BertForTokenClassification were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from datasets import load_from_disk

# Lade das vorbereitete Dataset
dataset_path = "Data_en"  # Pfad zu deinem Dataset
dataset = load_from_disk(dataset_path)



In [7]:
pip install 'accelerate>=0.26.0'


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 1.0 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
!pip install accelerate>=0.26.0
!pip install transformers[torch]


Defaulting to user installation because normal site-packages is not writeable


In [20]:
!pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable


In [22]:
!pip install 'accelerate>={ACCELERATE_MIN_VERSION}'

Defaulting to user installation because normal site-packages is not writeable


In [11]:
train_dataset='train_en'
eval_dataset='validation_en'
test_dataset='test_en'

In [15]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Erstelle den DataCollator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./results",             # Speichert das Modell hier
    evaluation_strategy="epoch",       # Evaluiert am Ende jeder Epoche
    save_strategy="epoch",             # Speichert am Ende jeder Epoche
    learning_rate=2e-5,                # Lernrate
    per_device_train_batch_size=16,    # Batchgröße fürs Training
    per_device_eval_batch_size=16,     # Batchgröße für Evaluation
    num_train_epochs=3,                # Anzahl der Epochen
    weight_decay=0.01,                 # Gewichtszerfall
    save_total_limit=2,                # Speichere nur die letzten zwei Checkpoints
    load_best_model_at_end=True,       # Lädt das beste Modell am Ende
    logging_dir="./logs",              # Log-Verzeichnis
    logging_steps=10,
    push_to_hub=False                  # Setze auf True, wenn du das Modell teilen möchtest
)


# Initialisiere den Trainer mit dem DataCollator
trainer = Trainer(
    model=model,
    args=training_args,         # Deine Trainingseinstellungen
    data_collator=data_collator, # Setze den DataCollator hier
    tokenizer=tokenizer,         # Übergib den Tokenizer
    train_dataset=train_dataset, # Dein Trainingsdatensatz
    eval_dataset=eval_dataset    # Dein Evaluationsdatensatz
)


/scratch/local/13438092/ipykernel_4073044/565325908.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()


AttributeError: 'str' object has no attribute 'keys'

In [ ]:
model.save_pretrained("./finetuned_wikineural")
tokenizer.save_pretrained("./finetuned_wikineural")
